In [2]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
help(tf.get_variable)

Help on function get_variable in module tensorflow.python.ops.variable_scope:

get_variable(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=None, collections=None, caching_device=None, partitioner=None, validate_shape=True, use_resource=None, custom_getter=None, constraint=None, synchronization=<VariableSynchronization.AUTO: 0>, aggregation=<VariableAggregation.NONE: 0>)
    Gets an existing variable with these parameters or create a new one.
    
    This function prefixes the name with the current variable scope
    and performs reuse checks. See the
    [Variable Scope How To](https://tensorflow.org/guide/variables)
    for an extensive description of how reusing works. Here is a basic example:
    
    ```python
    def foo():
      with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1])
      return v
    
    v1 = foo()  # Creates v.
    v2 = foo()  # Gets the same, existing v.
    assert v1 == v2
    ```
    
    If i

In [6]:
help(tf.placeholder)

Help on function placeholder in module tensorflow.python.ops.array_ops:

placeholder(dtype, shape=None, name=None)
    Inserts a placeholder for a tensor that will be always fed.
    
    **Important**: This tensor will produce an error if evaluated. Its value must
    be fed using the `feed_dict` optional argument to `Session.run()`,
    `Tensor.eval()`, or `Operation.run()`.
    
    For example:
    
    ```python
    x = tf.compat.v1.placeholder(tf.float32, shape=(1024, 1024))
    y = tf.matmul(x, x)
    
    with tf.compat.v1.Session() as sess:
      print(sess.run(y))  # ERROR: will fail because x was not fed.
    
      rand_array = np.random.rand(1024, 1024)
      print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.
    ```
    
    @compatibility(eager)
    Placeholders are not compatible with eager execution.
    @end_compatibility
    
    Args:
      dtype: The type of elements in the tensor to be fed.
      shape: The shape of the tensor to be fed (optional). If 

In [8]:
help(tf.square)

Help on function square in module tensorflow.python.ops.gen_math_ops:

square(x, name=None)
    Computes square of x element-wise.
    
    I.e., \\(y = x * x = x^2\\).
    
    Args:
      x: A `Tensor`. Must be one of the following types: `bfloat16`, `half`, `float32`, `float64`, `int32`, `int64`, `complex64`, `complex128`.
      name: A name for the operation (optional).
    
    Returns:
      A `Tensor`. Has the same type as `x`.
    
      If `x` is a `SparseTensor`, returns
      `SparseTensor(x.indices, tf.math.square(x.values, ...), x.dense_shape)`



In [7]:
class FfAgent(object):
    def __init__(self, session, input_size, output_size):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        
        self.observations_ph = tf.placeholder(dtype=tf.float32, shape=[None, self.input_size])
        # expected sum of discounted rewards
        self.esdr_ph = tf.placeholder(dtype=tf.float32, shape=[None, self.output_size])
        #self.reward_ph = tf.placeholder(dtype=tf.float32, shape=[None, 1])
        
        # Shared-parameter policy and value network
        W1 = tf.get_variable("w1", [self.input_size, 5])
        b1 = tf.get_variable("b1", [5])
        W2p = tf.get_variable("w2p", [5, self.output_size]) # policy
        b2p = tf.get_variable("b2p", [self.output_size])
        W2v = tf.get_variable("w2v", [5, 1]) # value
        b2v = tf.get_variable("b2v", [1])
        
        l1 = tf.nn.relu(tf.matmul(self.observations_ph, W1) + b1)
        # this will need to be changed to accommodate the range and character of action values
        l2p = tf.nn.relu(tf.matmul(l1, W2p) + b2p)
        l2v = tf.nn.relu(tf.matmul(l1, W2v) + b2v)
        
        self.loss = -1.*tf.reduce_sum(self.esdr_ph*tf.log(lwp)) + tf.reduce_sum(tf.square(l2v - self.esdr_ph))
        
        self.action_predictions = l2p
        self.esdr_predictions = l2v
        
    def trainSarBatches(self, sars):
        '''
        Expects sars to be a numpy array of shape [batch_size, 3]
        sars = [[state_t, action_t, reward_t+1], ...]
        '''
        